In [1]:
import sagemaker
import boto3
import pandas                as pd
import io
from sklearn.model_selection import train_test_split
from sklearn.utils           import resample

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/codezeus/.config/sagemaker/config.yaml


In [2]:
sm_boto3   = boto3.client("sagemaker")
s3_boto3   = boto3.client("s3")
sgm_sess   = sagemaker.Session()
get_region = sgm_sess.boto_session.region_name
bucket     = "your-financial-data-lake"

In [ ]:
# Get the data loation from the s3 bucket
s3_data_path = "s3://your-financial-data-lake/production/financial_summary/main-data.xlsx"
ml_path      = "s3://your-financial-data-lake/ML components/"

In [4]:
# tO VERIFY THE FILES IN THE S3 BUCKET

def verify_files_in_s3(bucket, prefix):
    response = s3_boto3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    if 'Contents' in response:
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("No files found in the specified bucket and prefix.")
    # if

# Verify
verify_files_in_s3(bucket, "production/financial_summary/")


production/financial_summary/main-data.xlsx
production/financial_summary/part-00000-52c36570-70e5-4796-8b0f-32d4aa8be5de-c000.snappy.parquet


In [5]:
# Read the data using pandas
s3_data  = s3_boto3.get_object(Bucket=bucket, Key='production/financial_summary/main-data.xlsx')
contents = io.BytesIO(s3_data['Body'].read()) 
data_df  = pd.read_excel(contents)

# See the data
data_df.head()

,company,year,revenue_old,netincome_old,netprofitmargin,grossprofit,revenue,netincome,costandexpenses,costofgoodsandservices,netproft,ebit,interest,ebt,proftaftertax,grossprofitmargin,YoY Revenue Change %,date,Sector
0,NFLX,2023,567.81,56.54,99.575562,5.536248e+07,56781000,56540000,2.592520e+06,1.418515e+06,4.477988e+07,5.276996e+07,87753.2726,5.268221e+07,7.902332e+06,97.501778,57745.608566,2023-10-25,Entertainment
1,TSLA,2023,305.68,78.39,256.444648,3.027479e+07,30568000,78390000,3.020022e+05,2.932118e+05,2.543271e+07,2.997279e+07,51950.6331,2.992084e+07,4.488125e+06,99.040788,107349.996729,2023-05-17,Automotive Technology
2,NVDA,2023,235.02,40.79,173.559697,2.314119e+07,23502000,40790000,1.003980e+06,3.608121e+05,1.880863e+07,2.213721e+07,9412.0812,2.212780e+07,3.319169e+06,98.464760,139655.403795,2023-09-01,Technology
3,AMZN,2023,318.72,76.35,239.551958,3.145261e+07,31872000,76350000,8.019831e+05,4.193853e+05,2.604707e+07,3.065063e+07,7020.1197,3.064361e+07,4.596542e+06,98.684157,102953.824674,2023-07-14,Consumer Discretionary
4,NVDA,2023,296.50,16.10,54.300169,2.958242e+07,29650000,16100000,6.506573e+05,6.757539e+04,2.452730e+07,2.893177e+07,76122.2536,2.885565e+07,4.328347e+06,99.772090,110676.779089,2023-01-08,Technology


In [6]:
# Remove the unnecessary columns
data_df = data_df.drop(columns=['revenue_old', 'netincome_old', 'YoY Revenue Change %'])
data_df

,company,year,netprofitmargin,grossprofit,revenue,netincome,costandexpenses,costofgoodsandservices,netproft,ebit,interest,ebt,proftaftertax,grossprofitmargin,date,Sector
0,NFLX,2023,99.575562,5.536248e+07,56781000,56540000,2.592520e+06,1.418515e+06,4.477988e+07,5.276996e+07,87753.2726,5.268221e+07,7.902332e+06,97.501778,2023-10-25,Entertainment
1,TSLA,2023,256.444648,3.027479e+07,30568000,78390000,3.020022e+05,2.932118e+05,2.543271e+07,2.997279e+07,51950.6331,2.992084e+07,4.488125e+06,99.040788,2023-05-17,Automotive Technology
2,NVDA,2023,173.559697,2.314119e+07,23502000,40790000,1.003980e+06,3.608121e+05,1.880863e+07,2.213721e+07,9412.0812,2.212780e+07,3.319169e+06,98.464760,2023-09-01,Technology
3,AMZN,2023,239.551958,3.145261e+07,31872000,76350000,8.019831e+05,4.193853e+05,2.604707e+07,3.065063e+07,7020.1197,3.064361e+07,4.596542e+06,98.684157,2023-07-14,Consumer Discretionary
4,NVDA,2023,54.300169,2.958242e+07,29650000,16100000,6.506573e+05,6.757539e+04,2.452730e+07,2.893177e+07,76122.2536,2.885565e+07,4.328347e+06,99.772090,2023-01-08,Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BABA,2022,86.010345,5.670410e+07,57807000,49720000,1.009134e+06,1.102900e+06,4.724994e+07,5.569497e+07,106800.3181,5.558817e+07,8.338225e+06,98.092100,2022-07-09,Consumer Discretionary
996,META,2020,188.776662,3.467294e+07,35444000,66910000,1.220932e+06,7.710630e+05,2.838080e+07,3.345200e+07,62829.3716,3.338918e+07,5.008376e+06,97.824560,2020-01-02,Communication Service
997,GOOG,2020,199.166193,5.148360e+06,5277000,10510000,2.357981e+05,1.286397e+05,4.172394e+06,4.912562e+06,3863.7970,4.908698e+06,7.363048e+05,97.562257,2020-10-29,Technology
998,BABA,2020,237.062230,2.126803e+07,21758000,51580000,9.366099e+05,4.899729e+05,1.726069e+07,2.033142e+07,24727.8388,2.030669e+07,3.046003e+06,97.748079,2020-01-14,Consumer Discretionary


In [7]:
sector_names  = data_df.Sector.unique()
company_names = data_df.company.unique()
years         = data_df.year.unique()
sector_names  = data_df.Sector.unique()

In [8]:
sector_cat = {
    'Entertainment': 0,
    'Automotive Technology': 1,
    'Technology':2,
    'Consumer Discretionary': 3,
    'Communication Service':4
}

In [9]:
def get_sector_category(sector):
    return sector_cat[sector]
#

data_df['Sector'] = data_df['Sector'].apply(lambda x: get_sector_category(x))

In [10]:
data_df

,company,year,netprofitmargin,grossprofit,revenue,netincome,costandexpenses,costofgoodsandservices,netproft,ebit,interest,ebt,proftaftertax,grossprofitmargin,date,Sector
0,NFLX,2023,99.575562,5.536248e+07,56781000,56540000,2.592520e+06,1.418515e+06,4.477988e+07,5.276996e+07,87753.2726,5.268221e+07,7.902332e+06,97.501778,2023-10-25,0
1,TSLA,2023,256.444648,3.027479e+07,30568000,78390000,3.020022e+05,2.932118e+05,2.543271e+07,2.997279e+07,51950.6331,2.992084e+07,4.488125e+06,99.040788,2023-05-17,1
2,NVDA,2023,173.559697,2.314119e+07,23502000,40790000,1.003980e+06,3.608121e+05,1.880863e+07,2.213721e+07,9412.0812,2.212780e+07,3.319169e+06,98.464760,2023-09-01,2
3,AMZN,2023,239.551958,3.145261e+07,31872000,76350000,8.019831e+05,4.193853e+05,2.604707e+07,3.065063e+07,7020.1197,3.064361e+07,4.596542e+06,98.684157,2023-07-14,3
4,NVDA,2023,54.300169,2.958242e+07,29650000,16100000,6.506573e+05,6.757539e+04,2.452730e+07,2.893177e+07,76122.2536,2.885565e+07,4.328347e+06,99.772090,2023-01-08,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BABA,2022,86.010345,5.670410e+07,57807000,49720000,1.009134e+06,1.102900e+06,4.724994e+07,5.569497e+07,106800.3181,5.558817e+07,8.338225e+06,98.092100,2022-07-09,3
996,META,2020,188.776662,3.467294e+07,35444000,66910000,1.220932e+06,7.710630e+05,2.838080e+07,3.345200e+07,62829.3716,3.338918e+07,5.008376e+06,97.824560,2020-01-02,4
997,GOOG,2020,199.166193,5.148360e+06,5277000,10510000,2.357981e+05,1.286397e+05,4.172394e+06,4.912562e+06,3863.7970,4.908698e+06,7.363048e+05,97.562257,2020-10-29,2
998,BABA,2020,237.062230,2.126803e+07,21758000,51580000,9.366099e+05,4.899729e+05,1.726069e+07,2.033142e+07,24727.8388,2.030669e+07,3.046003e+06,97.748079,2020-01-14,3


In [11]:
# # Calculate the YoY Revenue Change % for each company
# data_df.loc[data_df['company'].isin(['NFLX'])].groupby(by='year')['revenue'].sum()

In [12]:
sum_revenue = data_df.groupby(by=['company', 'year'])['revenue'].sum().reset_index()
sum_revenue = sum_revenue.rename(columns={'revenue': 'total_revenue'})
sum_revenue

,company,year,total_revenue
0,AAPL,2020,981440000
1,AAPL,2021,603766000
2,AAPL,2022,517062000
3,AAPL,2023,857634000
4,AMZN,2020,362054000
5,AMZN,2021,655169000
6,AMZN,2022,767109000
7,AMZN,2023,801786000
8,BABA,2020,599812000
9,BABA,2021,834049000


In [13]:
sum_revenue = sum_revenue.sort_values(by=['company','year'], ascending=False).reset_index().drop(columns=['index'])
sum_revenue

,company,year,total_revenue
0,TSLA,2023,943999000
1,TSLA,2022,879115000
2,TSLA,2021,678734000
3,TSLA,2020,707476000
4,ORCL,2023,824922000
5,ORCL,2022,783111000
6,ORCL,2021,970078000
7,ORCL,2020,786643000
8,NVDA,2023,617376000
9,NVDA,2022,742132000


In [14]:
# # Query the company first
# company = sum_revenue.loc[sum_revenue['company'] == 'TSLA']
# company

In [15]:
# company.loc[0, 'total_revenue']

In [16]:
# company.year.unique()

In [17]:
# company[(company['company'] == 'TSLA') & (company['year'] == 2023)]['total_revenue'].values[0]

In [18]:
sum_revenue.loc[(sum_revenue['company'] == 'TSLA') & (sum_revenue['year'] == 2023), 'total_revenue'].values[0]

943999000

In [19]:
def get_YoY_revenue_change_df(company_names):

    # Get the current year
    years = sum_revenue['year'].unique()
    for id_yr in range(len(years)-1):
        # get the current year
        current_revenue = sum_revenue.loc[(sum_revenue['company'] == company_names) & 
                                          (sum_revenue['year'] == years[id_yr]), 'total_revenue'].values[0]
        prev_revenue    = sum_revenue.loc[(sum_revenue['company'] == company_names) & 
                                          (sum_revenue['year'] == years[id_yr] - 1), 'total_revenue'].values[0]

        revenue_change = ((current_revenue - prev_revenue) / prev_revenue) * 100
                
        if id_yr == 0:
            sum_revenue.loc[(sum_revenue['company'] == company_names) & 
                            (sum_revenue['year'] == years[id_yr]), 'YoY Revenue Change %'] = 0.0
            sum_revenue.loc[(sum_revenue['company'] == company_names) & 
                            (sum_revenue['year'] == years[id_yr]-1), 'YoY Revenue Change %'] = revenue_change
        else:
            sum_revenue.loc[(sum_revenue['company'] == company_names) & 
                            (sum_revenue['year'] == years[id_yr]-1), 'YoY Revenue Change %'] = revenue_change
        # if
    # for

    return sum_revenue
#

In [20]:
for id_cmp in sum_revenue['company'].unique():
    print("Processing company:", id_cmp)

    see_result = get_YoY_revenue_change_df(id_cmp)

see_result

Processing company: TSLA
Processing company: ORCL
Processing company: NVDA
Processing company: NFLX
Processing company: MSFT
Processing company: META
Processing company: GOOG
Processing company: BABA
Processing company: AMZN
Processing company: AAPL


,company,year,total_revenue,YoY Revenue Change %
0,TSLA,2023,943999000,0.000000
1,TSLA,2022,879115000,7.380604
2,TSLA,2021,678734000,29.522759
3,TSLA,2020,707476000,-4.062611
4,ORCL,2023,824922000,0.000000
5,ORCL,2022,783111000,5.339090
6,ORCL,2021,970078000,-19.273399
7,ORCL,2020,786643000,23.318710
8,NVDA,2023,617376000,0.000000
9,NVDA,2022,742132000,-16.810487


In [21]:
sum_revenue

,company,year,total_revenue,YoY Revenue Change %
0,TSLA,2023,943999000,0.000000
1,TSLA,2022,879115000,7.380604
2,TSLA,2021,678734000,29.522759
3,TSLA,2020,707476000,-4.062611
4,ORCL,2023,824922000,0.000000
5,ORCL,2022,783111000,5.339090
6,ORCL,2021,970078000,-19.273399
7,ORCL,2020,786643000,23.318710
8,NVDA,2023,617376000,0.000000
9,NVDA,2022,742132000,-16.810487


In [22]:
data_df

,company,year,netprofitmargin,grossprofit,revenue,netincome,costandexpenses,costofgoodsandservices,netproft,ebit,interest,ebt,proftaftertax,grossprofitmargin,date,Sector
0,NFLX,2023,99.575562,5.536248e+07,56781000,56540000,2.592520e+06,1.418515e+06,4.477988e+07,5.276996e+07,87753.2726,5.268221e+07,7.902332e+06,97.501778,2023-10-25,0
1,TSLA,2023,256.444648,3.027479e+07,30568000,78390000,3.020022e+05,2.932118e+05,2.543271e+07,2.997279e+07,51950.6331,2.992084e+07,4.488125e+06,99.040788,2023-05-17,1
2,NVDA,2023,173.559697,2.314119e+07,23502000,40790000,1.003980e+06,3.608121e+05,1.880863e+07,2.213721e+07,9412.0812,2.212780e+07,3.319169e+06,98.464760,2023-09-01,2
3,AMZN,2023,239.551958,3.145261e+07,31872000,76350000,8.019831e+05,4.193853e+05,2.604707e+07,3.065063e+07,7020.1197,3.064361e+07,4.596542e+06,98.684157,2023-07-14,3
4,NVDA,2023,54.300169,2.958242e+07,29650000,16100000,6.506573e+05,6.757539e+04,2.452730e+07,2.893177e+07,76122.2536,2.885565e+07,4.328347e+06,99.772090,2023-01-08,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BABA,2022,86.010345,5.670410e+07,57807000,49720000,1.009134e+06,1.102900e+06,4.724994e+07,5.569497e+07,106800.3181,5.558817e+07,8.338225e+06,98.092100,2022-07-09,3
996,META,2020,188.776662,3.467294e+07,35444000,66910000,1.220932e+06,7.710630e+05,2.838080e+07,3.345200e+07,62829.3716,3.338918e+07,5.008376e+06,97.824560,2020-01-02,4
997,GOOG,2020,199.166193,5.148360e+06,5277000,10510000,2.357981e+05,1.286397e+05,4.172394e+06,4.912562e+06,3863.7970,4.908698e+06,7.363048e+05,97.562257,2020-10-29,2
998,BABA,2020,237.062230,2.126803e+07,21758000,51580000,9.366099e+05,4.899729e+05,1.726069e+07,2.033142e+07,24727.8388,2.030669e+07,3.046003e+06,97.748079,2020-01-14,3


# Sector Prediction

In [23]:
# Lets try to predict the sector 

# Drop the company and date columns
data_df = data_df.drop(columns=['company', 'date'])
data_df

,year,netprofitmargin,grossprofit,revenue,netincome,costandexpenses,costofgoodsandservices,netproft,ebit,interest,ebt,proftaftertax,grossprofitmargin,Sector
0,2023,99.575562,5.536248e+07,56781000,56540000,2.592520e+06,1.418515e+06,4.477988e+07,5.276996e+07,87753.2726,5.268221e+07,7.902332e+06,97.501778,0
1,2023,256.444648,3.027479e+07,30568000,78390000,3.020022e+05,2.932118e+05,2.543271e+07,2.997279e+07,51950.6331,2.992084e+07,4.488125e+06,99.040788,1
2,2023,173.559697,2.314119e+07,23502000,40790000,1.003980e+06,3.608121e+05,1.880863e+07,2.213721e+07,9412.0812,2.212780e+07,3.319169e+06,98.464760,2
3,2023,239.551958,3.145261e+07,31872000,76350000,8.019831e+05,4.193853e+05,2.604707e+07,3.065063e+07,7020.1197,3.064361e+07,4.596542e+06,98.684157,3
4,2023,54.300169,2.958242e+07,29650000,16100000,6.506573e+05,6.757539e+04,2.452730e+07,2.893177e+07,76122.2536,2.885565e+07,4.328347e+06,99.772090,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022,86.010345,5.670410e+07,57807000,49720000,1.009134e+06,1.102900e+06,4.724994e+07,5.569497e+07,106800.3181,5.558817e+07,8.338225e+06,98.092100,3
996,2020,188.776662,3.467294e+07,35444000,66910000,1.220932e+06,7.710630e+05,2.838080e+07,3.345200e+07,62829.3716,3.338918e+07,5.008376e+06,97.824560,4
997,2020,199.166193,5.148360e+06,5277000,10510000,2.357981e+05,1.286397e+05,4.172394e+06,4.912562e+06,3863.7970,4.908698e+06,7.363048e+05,97.562257,2
998,2020,237.062230,2.126803e+07,21758000,51580000,9.366099e+05,4.899729e+05,1.726069e+07,2.033142e+07,24727.8388,2.030669e+07,3.046003e+06,97.748079,3


In [24]:
# get columns
columns = list(data_df.columns)
columns

['year',
 'netprofitmargin',
 'grossprofit',
 'revenue',
 'netincome',
 'costandexpenses',
 'costofgoodsandservices',
 'netproft',
 'ebit',
 'interest',
 'ebt',
 'proftaftertax',
 'grossprofitmargin',
 'Sector']

In [25]:
# X_data = data_df[Feature_columns]
y_data = data_df["Sector"]

In [26]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data_df, test_size=0.2, random_state=42, stratify=y_data)

In [27]:
# We can use bootstrap sampling or class weights to handle the imbalance
# # USER INTERFACE
# bootstrap_iter = 10

# X_data_balanced, y_data_balanced = resample(X_data, y_data,
#                                             replace=True,
#                                             n_samples=len(y_data),
#                                             random_state=42)  

In [28]:
# See the shape of the test data
print("Shape of Xtest:", train_data.shape)
print("Shape of ytest:", test_data.shape)

Shape of Xtest: (800, 14)
Shape of ytest: (200, 14)


In [29]:
# Save the data into a csv file
train_data.to_csv('financial_train_data.csv', index=False)
test_data.to_csv('financial_test_data.csv', index=False)

In [30]:
# Load the csv data to s3 bucket
train_sk_prefix = "ML components/training-data"
test_sk_prefix  = "ML components/testing-data"

Alldata = {'train': train_data, 'test': test_data}

def upload_data_to_s3(data, bucket):
    for dataType in list(data.keys()):
        if dataType.endswith('train'):
            train_path = sgm_sess.upload_data('financial_train_data.csv', 
                                              bucket=bucket, 
                                              key_prefix=train_sk_prefix)
        else:
            test_path  =  sgm_sess.upload_data('financial_test_data.csv', 
                                               bucket=bucket, 
                                               key_prefix=test_sk_prefix)
        # if
    # for
    print("Data uploaded to S3 successfully.")

    return train_path, test_path
#
train_path, test_path = upload_data_to_s3(Alldata, bucket)


Data uploaded to S3 successfully.


In [ ]:
%%writefile ML_model_script.py

# OTHER IMPORTS
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import boto3
import joblib
import pathlib
import argparse
from os                     import path, environ
import pandas               as pd
# from imblearn.over_sampling import SMOTE
import joblib


def model_fn(model_dir):
    clf = joblib.load(path.join(model_dir, 'model.joblib'))
    return clf
#

def load_data(file_path):
    """
    Load and preprocess the training data.

    Input:
    ------
    file_path: Path to the directory containing the training data.

    Output:
    -------
    Returns:
        features: Features of the training dataset.
        label: Labels of the training dataset.
    """
    df       = pd.read_csv(file_path)
    features = df.iloc[:, :-1]
    label    = df.iloc[:, -1]
    return features, label
#


if __name__ == "__main__":
    
    print("[INFO] Extracting arguments....")
    parser = argparse.ArgumentParser()

    # Hyperparameters
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--max_depth', type=int, default=10)
    parser.add_argument('--min_samples_split', type=int, default=2) 
    parser.add_argument('--min_samples_leaf', type=int, default=1)
    parser.add_argument('--random_state', type=int, default=42)

    # Data, model and output directories
    parser.add_argument('--model-dir', type=str, default=environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file', type=str, default='financial_train_data.csv')
    parser.add_argument('--test-file', type=str, default='financial_test_data.csv')

    args, _ = parser.parse_known_args()

    print(args)

    print('SKLearn ersion:', sklearn.__version__)
    print('Joblib version:', joblib.__version__)

    print("[INFO] Loading the train and test data....")
    train_features, train_labels = load_data(path.join(args.train, args.train_file))
    test_features, test_labels   = load_data(path.join(args.test, args.test_file))

    # Using SMOTE to handle the imbalance
    # smote            = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=42)
    # x_smote, y_smote = smote.fit_resample(train_features, train_labels)

    print("Training Random Forest Classifier model....", flush=True)
    model = RandomForestClassifier(n_estimators=args.n_estimators,
    
                                    max_depth=args.max_depth,
                                    min_samples_split=args.min_samples_split,
                                    min_samples_leaf=args.min_samples_leaf,
                                    random_state=args.random_state)
    model.fit(train_features, train_labels)     
    print("Model trained successfully...",flush=True)

    model_path = path.join(args.model_dir, 'model.joblib')
    joblib.dump(model, model_path)
    print(f"Model persisted at {model_path}", flush=True)

    # Predicting the test data
    print("[INFO] Predicting the test data....", flush=True)
    ypred = model.predict(test_features)
    print("Prediction completed successfully.", flush=True)

    # Other metrics
    print("[INFO] Calculating the metrics....", flush=True)
    accuracy      = accuracy_score(test_labels, ypred)
    precision     = precision_score(test_labels, ypred, average='weighted')
    class_report  = classification_report(test_labels, ypred)   
    confusion_mat = confusion_matrix(test_labels, ypred)
    print("Metrics calculated successfully.", flush=True)


Overwriting ML_model_script.py


In [32]:
from sagemaker.sklearn.estimator import SKLearn

sklearn_est = SKLearn(
    entry_point='ML_model_script.py',
    role='arn:aws:iam::627206123828:role/service-role/AmazonSageMaker-ExecutionRole-20250809T175141',
    instance_type='ml.m5.xlarge',
    instance_count=1,
    framework_version='1.0-1',
    base_job_name='Financial-ml-model',
    py_version='py3',
    hyperparameters={
        'n_estimators': 100,
        'max_depth': 10,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'random_state': 42
    }, 
    use_spot_instances=True,
    max_wait = 7200,
    max_run = 3600,
)

In [33]:
# Lauch the training job
print("Starting the training job...")
sklearn_est.fit({"train":train_path, "test":test_path},wait=True)

Starting the training job...


INFO:sagemaker:Creating training-job with name: Financial-ml-model-2025-08-16-23-17-24-851


2025-08-16 23:17:27 Starting - Starting the training job...
2025-08-16 23:18:01 Downloading - Downloading input data...
2025-08-16 23:18:22 Downloading - Downloading the training image.....2025-08-16 23:19:18,759 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-08-16 23:19:18,763 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-16 23:19:18,766 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-08-16 23:19:18,781 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-08-16 23:19:19,008 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-16 23:19:19,012 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-08-16 23:19:19,031 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-16 23:19:19,034 sagemaker-training-toolkit IN

In [34]:
# To see more information regadding training

sklearn_est.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_est.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2025-08-16 23:19:35 Starting - Preparing the instances for training
2025-08-16 23:19:35 Downloading - Downloading the training image
2025-08-16 23:19:35 Training - Training image download completed. Training in progress.
2025-08-16 23:19:35 Uploading - Uploading generated training model
2025-08-16 23:19:35 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-2-627206123828/Financial-ml-model-2025-08-16-23-17-24-851/output/model.tar.gz


In [35]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = 'Custom-sklearn-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model      = SKLearnModel(
    name= model_name,
    model_data=artifact,
    role='arn:aws:iam::627206123828:role/service-role/AmazonSageMaker-ExecutionRole-20250809T175141',
    entry_point="ML_model_script.py",
    py_version='py3',
    framework_version='1.0-1',
)

In [36]:
# Saving into a separate folder to deplot it as an end point

# To deploy it
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2025-08-16-23-19-54


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2025-08-16-23-19-54
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2025-08-16-23-19-54
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2025-08-16-23-19-54


-----!

In [38]:
test_data[columns[:-1]][0:2].values.tolist()

[[2021.0,
  55.275202441028,
  8299244.5823,
  8521000.0,
  4710000.0,
  153396.671,
  221755.4177,
  6916557.7751,
  8145847.9113,
  8721.1171,
  8137126.7942,
  1220569.0191,
  97.3975423342331],
 [2020.0,
  208.75259543204,
  18395791.1068,
  18783000.0,
  39210000.0,
  246152.5243,
  387208.8932,
  15407538.1879,
  18149638.5825,
  23123.0673,
  18126515.5152,
  2718977.3273,
  97.9385141180855]]

In [39]:
print(predictor.predict(test_data[columns[:-1]][0:2].values.tolist()))

[2 2]


In [ ]:
# to delete endpoint

sm_boto3.delete_endpoint(EndpointName = endpoint_name)

{'ResponseMetadata': {'RequestId': 'd4074161-6404-459b-9a51-6708a0de9d6d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd4074161-6404-459b-9a51-6708a0de9d6d',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 16 Aug 2025 23:32:40 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}